<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Content Based</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Popular
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])
ratings = ratings.drop(['timestamp'],axis=1)
ratings.head(5)

,user,item,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


# Unary ratings

In [2]:
unary = ratings.copy()
unary['rating'] = unary['rating'].replace([1,2,3,4,5],1)
unary

,user,item,rating
0,196,242,1
1,186,302,1
2,22,377,1
3,244,51,1
4,166,346,1
...,...,...,...
99995,880,476,1
99996,716,204,1
99997,276,1090,1
99998,13,225,1


# Genres Matrix

In [3]:
m_cols = ['movie id','movie title','release date','video release date','IMDb URL',
          'unknown','Action','Adventure','Animation','Children','Comedy','Crime',
          'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
          'Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 100k\\u.item', sep='|', names=m_cols,encoding='latin-1')
movies = movies.set_index('movie id')
movies = movies.drop(['video release date'],axis = 1)
genres = movies.drop(['movie title','release date','IMDb URL'],axis = 1)
genres

,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


# Unpivot Ratings Matrix

In [5]:
ratings = ratings.pivot(index = 'user',columns='item',values ='rating')
ratings = ratings.fillna(0)
ratings

item,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Unpivot Unary matrix

In [6]:
unary = unary.pivot(index = 'user',columns='item',values ='rating')
unary = unary.fillna(0)
unary

item,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Profile matrix

In [7]:
np.seterr(divide='ignore', invalid='ignore')
dot = unary.values.dot(genres.values)*5
dot = pd.DataFrame(dot)
profile = pd.DataFrame((ratings).values.dot(genres.values), columns=genres.columns, index=(ratings).index)
profile = profile.values/(dot.values)
profile = pd.DataFrame(profile)
profile.columns = genres.columns
profile = profile.fillna(0)
profile

,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.8,0.660550,0.577512,0.685956,0.423689,0.700691,0.689827,0.96,0.795254,0.707180,1.00000,0.673638,0.562773,0.749195,0.774714,0.805355,0.730015,0.730087,0.747961
1,0.0,0.740712,0.854066,0.800000,0.589898,0.761870,0.714972,0.00,0.763481,0.600000,0.89282,0.600000,0.600000,0.690616,0.816781,0.724967,0.702932,0.688316,0.000000
2,0.0,0.546117,0.703449,0.000000,0.000000,0.510883,0.602632,1.00,0.590857,0.000000,0.50718,0.489238,0.400000,0.616849,0.659199,0.529391,0.495490,0.569028,0.000000
3,0.0,0.778578,0.683036,0.000000,0.000000,1.000000,0.944949,1.00,0.902923,0.000000,0.00000,0.800000,1.000000,0.831699,0.879104,0.767110,0.805761,0.894427,0.000000
4,0.8,0.617892,0.638904,0.755677,0.475428,0.598341,0.786087,0.00,0.496808,0.500000,1.00000,0.482704,0.662493,0.590651,0.430393,0.705344,0.575084,0.639139,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.0,0.801035,0.675286,0.800000,0.800000,0.903261,0.875959,0.00,0.910390,0.800000,0.00000,0.600000,0.800000,0.800000,0.964828,0.813681,0.816329,1.000000,0.000000
939,0.0,0.692179,0.601980,0.892820,0.834023,0.696632,0.866942,0.00,0.711386,0.000000,0.80000,0.627468,0.581807,0.854066,0.707142,0.613916,0.676848,0.655002,0.000000
940,0.0,0.763372,0.777476,0.944165,0.912702,0.806064,0.600000,0.00,0.834315,0.000000,0.00000,0.000000,0.800000,1.000000,1.000000,0.782087,0.807326,1.000000,0.000000
941,0.0,0.821879,0.941015,0.940122,0.855319,0.805435,0.000000,0.00,0.888006,0.785641,1.00000,0.754692,0.873310,0.802821,0.880546,0.823213,0.797888,0.937058,0.907180


# Compute Predictions (Genres . Profile^T)

In [8]:
count = genres.sum(axis=1)
count = (pd.DataFrame(count))
prediction = pd.DataFrame((genres.values.dot((profile.T).values))/count.values, columns=(profile.T).columns, index=genres.index)
prediction = prediction.T
prediction.columns = ratings.columns
prediction.index = ratings.index
prediction

item,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,0.603445,0.656026,0.730015,0.718832,0.738365,0.795254,0.800304,0.639878,0.795254,0.762671,...,0.695282,0.795254,0.795254,0.795254,0.795254,0.795254,0.752364,0.784984,0.700691,0.795254
2,0.717256,0.765903,0.702932,0.755354,0.727128,0.763481,0.744224,0.705083,0.763481,0.725898,...,0.721822,0.763481,0.763481,0.763481,0.763481,0.763481,0.759857,0.790131,0.761870,0.763481
3,0.170294,0.581685,0.495490,0.549286,0.562993,0.590857,0.560124,0.367247,0.590857,0.579942,...,0.520804,0.590857,0.590857,0.590857,0.590857,0.590857,0.577344,0.625028,0.510883,0.590857
4,0.333333,0.755792,0.805761,0.893834,0.884544,0.902923,0.835017,0.634308,0.902923,0.898675,...,0.792169,0.902923,0.902923,0.902923,0.902923,0.902923,0.842433,0.891014,1.000000,0.902923
5,0.609816,0.610627,0.575084,0.571014,0.619326,0.496808,0.601076,0.523526,0.496808,0.567974,...,0.596488,0.496808,0.496808,0.496808,0.496808,0.496808,0.502738,0.463601,0.598341,0.496808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.834420,0.764217,0.816329,0.871562,0.867559,0.910390,0.862036,0.871217,0.910390,0.955195,...,0.808682,0.910390,0.910390,0.910390,0.910390,0.910390,0.890579,0.937609,0.903261,0.910390
940,0.807825,0.657003,0.676848,0.700066,0.751725,0.711386,0.662651,0.747347,0.711386,0.683194,...,0.684514,0.711386,0.711386,0.711386,0.711386,0.711386,0.691995,0.709264,0.696632,0.711386
941,0.887644,0.782724,0.807326,0.801250,0.747213,0.834315,0.808201,0.851027,0.834315,0.917157,...,0.785349,0.834315,0.834315,0.834315,0.834315,0.834315,0.903663,0.917157,0.806064,0.834315


# Reset to 1-5 star scale

In [9]:
OldMin = 0
OldMax = 1
NewMin = 1
NewMax = 5
prediction = (((prediction - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin
prediction

item,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,3.413781,3.624103,3.920059,3.875326,3.953461,4.181016,4.201217,3.559512,4.181016,4.050682,...,3.781130,4.181016,4.181016,4.181016,4.181016,4.181016,4.009458,4.139936,3.802762,4.181016
2,3.869023,4.063614,3.811730,4.021416,3.908513,4.053924,3.976896,3.820331,4.053924,3.903594,...,3.887288,4.053924,4.053924,4.053924,4.053924,4.053924,4.039427,4.160524,4.047478,4.053924
3,1.681177,3.326741,2.981958,3.197143,3.251971,3.363429,3.240497,2.468987,3.363429,3.319770,...,3.083214,3.363429,3.363429,3.363429,3.363429,3.363429,3.309377,3.500112,3.043531,3.363429
4,2.333333,4.023166,4.223043,4.575335,4.538177,4.611692,4.340066,3.537231,4.611692,4.594700,...,4.168678,4.611692,4.611692,4.611692,4.611692,4.611692,4.369730,4.564055,5.000000,4.611692
5,3.439262,3.442507,3.300334,3.284055,3.477305,2.987233,3.404304,3.094103,2.987233,3.271895,...,3.385952,2.987233,2.987233,2.987233,2.987233,2.987233,3.010953,2.854402,3.393363,2.987233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.337681,4.056867,4.265318,4.486247,4.470238,4.641559,4.448142,4.484868,4.641559,4.820780,...,4.234728,4.641559,4.641559,4.641559,4.641559,4.641559,4.562314,4.750435,4.613044,4.641559
940,4.231301,3.628010,3.707392,3.800264,4.006901,3.845546,3.650605,3.989389,3.845546,3.732776,...,3.738055,3.845546,3.845546,3.845546,3.845546,3.845546,3.767981,3.837058,3.786528,3.845546
941,4.550575,4.130897,4.229302,4.205001,3.988853,4.337258,4.232804,4.404108,4.337258,4.668629,...,4.141394,4.337258,4.337258,4.337258,4.337258,4.337258,4.614651,4.668629,4.224258,4.337258


In [10]:
print(prediction.min().min())
print(prediction.max().max())

1.0
5.000000000000001


In [11]:
ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])
ratings = ratings.drop(['timestamp'],axis=1)
ratings = ratings.pivot(index = 'user',columns='item',values ='rating')
ratings

item,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
MAE = abs(ratings-prediction)
MAE

item,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,1.586219,0.624103,0.079941,0.875326,0.953461,0.818984,0.201217,2.559512,0.818984,1.050682,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.130977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.903594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.560738,0.442507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.358441,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,1.800264,NaN,NaN,0.349395,1.010611,0.845546,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,0.449425,NaN,NaN,NaN,NaN,NaN,0.232804,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
MAE.mean(axis=1).mean()

0.7615416851863587

In [14]:
RMSE = ((((MAE**2).mean(axis=1)))**(1/2)).mean()
RMSE

0.9647800698796086